### Setup

In [ ]:
from common import *

figure_saver = figure_saver(sub_directory="interp_comp")

## Create datasets

In [ ]:
variables = ("SWI(1)", "FAPAR", "LAI", "VOD Ku-band", "SIF")
datasets = Datasets(
    (Copernicus_SWI(), MOD15A2H_LAI_fPAR(), VODCA(), GlobFluo_SIF())
).select_variables(variables)

### Temporal interpolation

In [ ]:
timeperiod = (datetime(2010, 1, 1, 0, 0), datetime(2015, 1, 1, 0, 0))
orig_datasets = datasets.copy()
for dataset in orig_datasets:
    datasets.add(dataset.get_temporally_interpolated_dataset(timeperiod, 3))
for dataset in datasets:
    dataset.limit_months(*timeperiod)

In [ ]:
datasets

In [ ]:
datasets.show()

## Compare original to interpolated datasets

In [ ]:
period_str = f"{timeperiod[0]:%Y-%m} - {timeperiod[1]:%Y-%m}"

saver = figure_saver(sub_directory="interp_comp")
for var in variables:
    interp_var = var + f" {n_months}NN"
    plot_vars = (var, interp_var)
    plot_cubes = [
        datasets.select_variables(plot_var, inplace=False).cube
        for plot_var in plot_vars
    ]

    for plot_var, plot_cube in zip(plot_vars, plot_cubes):

        if not plot_cube.coords("month_number"):
            iris.coord_categorisation.add_month_number(plot_cube, "time")

        fig = cube_plotting(plot_cube, title=f"{plot_var}\n{period_str}")
        saver.save_figure(fig, f"{plot_var} mean")

    replace_cube_coord(plot_cubes[0], plot_cubes[1].coord("month_number"))
    replace_cube_coord(plot_cubes[0], plot_cubes[1].coord("latitude"))
    replace_cube_coord(plot_cubes[0], plot_cubes[1].coord("longitude"))

    mean_1 = np.mean(plot_cubes[1].data, axis=0)
    mean_0 = np.mean(plot_cubes[0].data, axis=0)

    fig = cube_plotting(
        mean_1 - mean_0,
        title=f"{plot_var} <interp> - <normal>\n{period_str}",
        cmap_midpoint=0,
        cmap_symmetric=True,
    )
    saver.save_figure(fig, f"{var} interp - normal mean")

    fig = cube_plotting(
        100 * (mean_1 - mean_0) / mean_0,
        title=f"{plot_var} (<interp> - <normal>) / <normal> (%)\n{period_str}",
        cmap_midpoint=0,
        cmap_symmetric=True,
        colorbar_kwargs={"label": "%"},
        log=True,
    )
    saver.save_figure(fig, f"{var} (interp - normal) normalised mean")

In [ ]:
period_str = f"{timeperiod[0]:%Y-%m} - {timeperiod[1]:%Y-%m}"

saver = figure_saver(sub_directory="interp_comp_hist")

for var in variables:
    interp_var = var + f" {n_months}NN"
    plot_vars = (var, interp_var)
    plot_cubes = [
        datasets.select_variables(plot_var, inplace=False).cube
        for plot_var in plot_vars
    ]

    for plot_var, plot_cube in zip(plot_vars, plot_cubes):

        if not plot_cube.coords("month_number"):
            iris.coord_categorisation.add_month_number(plot_cube, "time")

    replace_cube_coord(plot_cubes[0], plot_cubes[1].coord("month_number"))
    replace_cube_coord(plot_cubes[0], plot_cubes[1].coord("latitude"))
    replace_cube_coord(plot_cubes[0], plot_cubes[1].coord("longitude"))

    mean_1 = np.mean(plot_cubes[1].data, axis=0)
    mean_0 = np.mean(plot_cubes[0].data, axis=0)

    fig, ax = plt.subplots()
    ax.hist((mean_1 - mean_0).flatten(), bins=200)
    ax.set_title(f"{plot_var} <interp> - <normal>\n{period_str}")
    ax.set_yscale("log")
    ax.set_xlabel(f"{plot_var}")
    saver.save_figure(fig, f"{var} interp - normal hist")

    fig, ax = plt.subplots()
    ax.hist((100 * (mean_1 - mean_0) / mean_0).flatten(), bins=200)
    ax.set_title(f"{plot_var} (<interp> - <normal>) / <normal> (%)\n{period_str}")
    ax.set_yscale("log")
    ax.set_xlabel("%")
    saver.save_figure(fig, f"{var} (interp - normal) normalised hist")

In [ ]:
period_str = f"{timeperiod[0]:%Y-%m} - {timeperiod[1]:%Y-%m}"

saver = figure_saver(sub_directory="interp_comp_timeseries")

for var in variables:
    interp_var = var + f" {n_months}NN"
    plot_vars = (var, interp_var)
    plot_cubes = [
        datasets.select_variables(plot_var, inplace=False).cube
        for plot_var in plot_vars
    ]

    for plot_var, plot_cube in zip(plot_vars, plot_cubes):
        if not plot_cube.coords("month_number"):
            iris.coord_categorisation.add_month_number(plot_cube, "time")

    for coord in ("time", "latitude", "longitude"):
        assert np.all(
            plot_cubes[0].coord(coord).points == plot_cubes[1].coord(coord).points
        )

    replace_cube_coord(plot_cubes[0], plot_cubes[1].coord("month_number"))
    replace_cube_coord(plot_cubes[0], plot_cubes[1].coord("latitude"))
    replace_cube_coord(plot_cubes[0], plot_cubes[1].coord("longitude"))

    mean_1 = np.mean(plot_cubes[1].data, axis=0)
    mean_0 = np.mean(plot_cubes[0].data, axis=0)

    normal_data = plot_cubes[0].data
    interp_data = plot_cubes[1].data

    mean_diff_data = (mean_1 - mean_0) / mean_0

    unmasked = get_unmasked(mean_diff_data)
    sort_indices = np.argsort(np.abs(unmasked))
    for diff_i in sort_indices[-20:]:
        diff = unmasked[diff_i]
        indices = np.where(mean_diff_data == diff)

        lat = plot_cubes[0].coord("latitude").points[indices[0][0]]
        lon = plot_cubes[0].coord("longitude").points[indices[1][0]]

        lat_str = f"{abs(lat):0.1f}" + ("°N" if lat >= 0 else "°S")
        lon_str = f"{abs(lon):0.1f}" + ("°W" if lon >= 0 else "°E")
        loc_str = f"{lat_str}, {lon_str}"

        normal_data_sel = normal_data[:, indices[0][0], indices[1][0]]
        interp_data_sel = interp_data[:, indices[0][0], indices[1][0]]

        fig, ax = plt.subplots()
        ax.set_title(f"{plot_var}, {diff:0.1f}\n{period_str}, {loc_str}")
        ax.plot(normal_data_sel, marker="o", label="Original")
        ax.plot(interp_data_sel, marker="x", linestyle="--", label="Interpolated")
        ax.legend(loc="best")
        saver.save_figure(fig, f"{var}_{lat:0.1f}_{lon:0.1f}")

In [ ]:
cube_plotting(mean_diff_data.mask)